# 인지검사별 피처 분석

In [10]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

In [11]:
BASE_DIR = "../data"
MODEL_SAVE_DIR = "./model"
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

FEATURE_SAVE_PATH = os.path.join(BASE_DIR, "all_train_data.feather")

In [12]:
try:
    print("전처리된 메인 피처 로드 중...")
    df = pd.read_feather(FEATURE_SAVE_PATH)
except FileNotFoundError as e:
    print(f"경고: {e}")
    print("먼저 1_Preprocess_delta_logratio.py를 실행하여 all_train_data.feather를 생성해야 합니다.")
    raise
print("데이터 로드 완료.")

전처리된 메인 피처 로드 중...


ArrowKeyError: No type extension with name arrow.py_extension_type found

In [ ]:
# ------------------------------------------------------
# 1. 주요 피처 목록 스캔
# ------------------------------------------------------
rt_cols = [c for c in df.columns if c.endswith("rt_mean") or c.endswith("rt_std")]
acc_cols = [c for c in df.columns if "acc" in c]

cv_cols = [c for c in df.columns if c.endswith("rt_cv")]
tradeoff_cols = [c for c in df.columns if "speed" in c and "acc" in c]

logratio_cols = [c for c in df.columns if "log_ratio" in c or "logratio" in c]

# Stroop / Flanker / Attention 관련 조건 비교 후보
stroop_cols = [c for c in df.columns if "A4" in c and ("congruent" in c or "incongruent" in c)]
flanker_cols = [c for c in df.columns if "B4" in c and ("congruent" in c or "incongruent" in c)]
attention_cols = [c for c in df.columns if "A3" in c and ("valid" in c or "invalid" in c)]

change_cols = [c for c in df.columns if ("B1" in c or "B2" in c) and ("change" in c or "nonchange" in c)]

In [ ]:
# ------------------------------------------------------
# Helper: 기초 통계 테이블 생성
# ------------------------------------------------------
def make_stats_table(cols, title):
    rows = []
    for col in cols:
        if col not in df.columns:
            print(f"{col} 없음 → 확실하지 않음")
            continue
        s = df[col].dropna()
        if s.empty:
            continue
        rows.append({
            "feature": col,
            "count": s.count(),
            "mean": s.mean(),
            "std": s.std(),
            "min": s.min(),
            "25%": s.quantile(0.25),
            "50%": s.quantile(0.5),
            "75%": s.quantile(0.75),
            "max": s.max(),
            "skewness": skew(s),
            "kurtosis": kurtosis(s)
        })
    print(f"\n=== {title} ===")
    display(pd.DataFrame(rows))

In [ ]:
# ------------------------------------------------------
# Helper: IQR 기반 이상치 탐지
# ------------------------------------------------------
def detect_outliers(cols, title):
    report = []
    for col in cols:
        if col not in df.columns:
            continue
        s = df[col].dropna()
        Q1, Q3 = s.quantile(0.25), s.quantile(0.75)
        IQR = Q3 - Q1
        lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
        count = ((s < lower) | (s > upper)).sum()
        report.append({
            "feature": col,
            "outlier_count": count,
            "lower_bound": lower,
            "upper_bound": upper
        })
    print(f"\n=== {title} - IQR Outlier 탐지 ===")
    display(pd.DataFrame(report))

- **반응시간/정확도 분포**
   - A1/A2(속도·정지 예측), A3(주의 전환), A4(Stroop), B3(신호등), B4(Flanker) 등 
 	핵심 과제의 `*_rt_mean`, `*_rt_std`, `*_acc_*` 분포 확인
   - 이상치(outlier) 후보(극단적 반응시간, 지나치게 낮은 정확도)를 확인하여 
 	전처리/모델링에서의 처리 방식을 검토

- **조건 간 차이 및 log-ratio**
   - Stroop/Flanker 계열
 	- 조건별 평균/정확도 차이 및 log-ratio(`A4_stroop_*`, `B4_flanker_*`)를 통해 
   	간섭 효과 크기가 어느 정도 범위에서 움직이는지 확인
   - 주의 전환/시야각 계열
 	- `A3_valid_acc` vs `A3_invalid_acc`, `B1/B2_change_acc` vs `nonchange_acc` 등 
   	조건별 난이도 차이가 일관되게 나타나는지 검토

 - **Speed–Accuracy 및 CV**
   - `*_speed_acc_tradeoff`, `*_rt_cv` 분포를 통해
 	- 빠른 대신 오류가 많은 유형
 	- 느리지만 정확한 유형 
 	등을 구분할 수 있는지, 값 범위가 안정적인지 확인
